In [1]:
%matplotlib inline

In [2]:
import pandas as pd
import matplotlib as plt
import os

Before proceed to calculating the missing data percentage in the systems data sets, they need to be cleaned a little. Due to the way of extracting the data - writing data for each year on same file after the previous, leads to occurrence of headers before every year's first record. The unnecessary header rows will be dropped using `delete_additional_header_lines`. 

The data will be grouped by year and the percentage of missing data is calculated with `missing_values_percent`. In order to group the data by year, the timestamp feature needs to be assigned to `datetime64` type. That's accomplished with `convert_data_types`. 

In [3]:
def delete_additional_header_lines(dataframe):
    '''
    Deletes additional header lines in the dataframe.
    '''
    dataframe = dataframe.drop(dataframe[dataframe['SiteID'] == 'SiteID'].index)
    return dataframe

In [4]:
def convert_data_types(dataframe, columns_type: dict):
    '''
    Changes the data types in dataframe according to the one provided in columns_type dictionary. 
    The parameter columns_type should have the format {column_name: new data type}. 
    '''
    for column, column_type in columns_type.items():
        try:
            if columns_type[column] != dataframe[column].dtypes:
                dataframe[column] = dataframe[column].astype(columns_type[column])
        except ValueError:
            dataframe[column] = dataframe[column].astype(float)
    return dataframe

In [5]:
def missing_values_percent(dataframe, year_column):
    '''
    Group data by year and calculate the percentage of missing values per year for every feature in the data frame.
    Returns dataframe.
    '''
    missing_values = dataframe.groupby(dataframe[year_column].dt.year).apply(lambda x: (x.isna().sum() / (x.size / dataframe.shape[1])) * 100)
    return missing_values

All data files extracted from 'Annual Data CSV for a System' of NREL API is saved in `annual_data` folder. From that folder all filenames will be assigned to `files` list.
There are total of 30 files, most of the with more than 300 hundred records, they can't be loaded simultaneously. Because of that each file will be loaded as `pv_data_system` data frame. After the necessary processing the missing data percentage will be calculated as `missing_values_report` and concatenated to the `report_data`. At the end `Report_data`, which contains the missing data percentage for each year for every system will be exported as .csv file for further examination. 

In [6]:
path = os.path.join(os.getcwd(),'annual_data')

files = [os.path.join(path,i) for i in os.listdir(path) if os.path.isfile(os.path.join(path,i))]

In [7]:
report_data = pd.DataFrame()

In [8]:
for file in files:

    pv_data_system = pd.read_csv(file, low_memory = False)
    
    pv_data_system = delete_additional_header_lines(pv_data_system)
    
    column_types = {'Date-Time': 'datetime64[ns]'}
    pv_data_system = convert_data_types(pv_data_system, column_types)
    
    missing_values_report = missing_values_percent(pv_data_system, 'Date-Time')
    missing_values_report.insert(loc=0, column = 'system', value = pv_data_system['SiteID'])
    
    report_data = pd.concat([report_data, missing_values_report])
    pv_data_system = pd.DataFrame()

In [9]:
report_data

,system,SiteID,Date-Time,ac_power,ambient_temp,dc_power,inv1_ac_power,inv1_dc_current,inv1_dc_power,inv1_dc_voltage,...,ac_meter_2_power,gh_irradiance,module_temp_south_F,module_temp_west_F,poa_irradiance_south_array,poa_irradiance_west_array,inv4_dc_temp,inv7_temp,ambient_temp_1_F,ambient_temp_2_F
Date-Time,,,,,,,,,,,,,,,,,,,,,
2011,1204,0.0,0.0,0.000000,0.048961,23.814929,0.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,1204,0.0,0.0,0.000000,0.000000,0.136655,0.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2013,1204,0.0,0.0,3.052915,0.003685,4.200752,0.0,0.0,3.052915,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2014,1204,0.0,0.0,0.000000,0.244418,99.988974,0.0,0.0,0.000000,0.000000,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2015,1204,0.0,0.0,98.730483,0.308550,0.000000,0.0,0.0,0.000000,0.345725,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
2007,1230,0.0,0.0,NaN,NaN,0.000000,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.182119,0.0
2011,1220,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2012,1220,0.0,0.0,0.000000,0.000000,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [10]:
report_data.to_csv('missing_data_report')